Communications Operations Procedure for Proximity Links (COP-P)

In [1]:
from enum import Enum, auto
from dataclasses import dataclass, field
from collections import deque

MOD = 256

# ---------- Modulo-256 helpers (8‑bit counters) ----------

def mod_diff(a, b):
    """A - B in 'number of increments from B to reach A' (mod 256)."""
    return (a - b) % MOD

def less_than(b, a):
    """B < A using COP‑P rule."""
    d = mod_diff(a, b)
    return 1 <= d <= 127

def greater_than(b, a):
    """B > A using COP‑P rule."""
    d = mod_diff(a, b)
    return 128 <= d <= 255

def equal(a, b):
    """B == A using COP‑P rule."""
    return mod_diff(a, b) == 0

# ---------- Simple frame model ----------

@dataclass
class Frame:
    seq: int = 0
    expedited: bool = False
    payload: bytes = b""

# ---------- FOP‑P (Sending side) ----------

class FOPState(Enum):
    S1_ACTIVE = auto()
    S2_RESYNC = auto()

@dataclass
class FOPP:
    # Queues
    exp_queue: deque = field(default_factory=deque)   # Expedited
    seq_queue: deque = field(default_factory=deque)   # New sequence-controlled
    sent_queue: deque = field(default_factory=deque)  # Sent but unacknowledged

    # FOP‑P variables (6.2.3.1.1 and 6.2.3.1.2)
    VE_S: int = 0      # VE(S)
    V_S: int = 0       # V(S)
    VV_S: int = 0      # VV(S)
    N_R: int = 0       # N(R) from current PLCW
    NN_R: int = 0      # NN(R) previous PLCW report value
    R_R: bool = False  # R(R) from current PLCW
    RR_R: bool = False # RR(R) from previous PLCW

    NEED_PLCW: bool = True
    NEED_STATUS_REPORT: bool = True

    SYNCH_TIMER: int = 0
    RESYNC: bool = False

    Transmission_Window: int = 10   # MIB parameter
    Resync_Local: bool = True       # MIB parameter
    Synch_Timeout: int = 5          # MIB parameter (example default)

    state: FOPState = FOPState.S1_ACTIVE

    # ----- General procedures (6.2.3.1.x) -----

    def initialize(self):
        """SE0: Initialize."""
        self.V_S = self.VE_S = self.VV_S = self.NN_R = self.N_R = 0
        self.R_R = self.RR_R = self.RESYNC = False
        self.NEED_PLCW = self.NEED_STATUS_REPORT = True
        self.sent_queue.clear()
        self.seq_queue.clear()
        self.exp_queue.clear()
        self.SYNCH_TIMER = 0
        self.state = FOPState.S1_ACTIVE

    def remove_acknowledged_frames(self):
        """Remove n frames from Sent queue, where n = N(R) − NN(R)."""
        n = mod_diff(self.N_R, self.NN_R)
        # _ is a throwaway variable
        for _ in range(min(n, len(self.sent_queue))):
            self.sent_queue.popleft()

    def start_synch_timer(self):
        if self.SYNCH_TIMER == 0:
            self.SYNCH_TIMER = self.Synch_Timeout
            # MIB Parameter Synch_Timeout

    def clear_synch_timer(self):
        self.SYNCH_TIMER = 0

    def tick_timer(self):
        """
        Call this from a scheduler.
        Returns 'SE6' when Synch_Timer expires (triggers local resync).
        """
        if self.SYNCH_TIMER > 0:
            self.SYNCH_TIMER -= 1
            if self.SYNCH_TIMER == 1:
                # Expiry at next tick; indicate SE6 now.
                return "SE6"
        return None

    def store_plcw(self):
        """Store current PLCW info into NN(R), RR(R)."""
        self.NN_R = self.N_R
        self.RR_R = self.R_R

    # ----- Frame sending procedures (6.2.3.1.6–1.8) -----

    def send_exp_frame(self):
        """"
        Remove frame from EXP queue
        Assign VE(S) to the frame
        Increment VE(S)
        Report VE(S) to the I/O Sublayer
        Transfer this frame to the Frame Sublayer
        """
        """Send EXP frame procedure."""
        if not self.exp_queue:
            return None
        frame = self.exp_queue.popleft()
        frame.expedited = True
        frame.seq = self.VE_S
        self.VE_S = (self.VE_S + 1) % MOD
        # Report VE(S) to I/O sublayer could be done via callback
        return frame

    def resend_seq_frame(self):
        """
        Copy frame number VV(S) from the Sent queue
        Increment VV(S)
        Transfer this frame to the Frame Sublayer
        """

        """
        Resend SEQ frame (frame number VV(S) from Sent queue).
        Here, 'frame number' = relative index from NN(R) for simplicity.
        """
        # Compute index relative to oldest unacknowledged frame NN(R)
        idx = mod_diff(self.VV_S, self.NN_R)
        if 0 <= idx < len(self.sent_queue):
            frame = self.sent_queue[idx]
            self.VV_S = (self.VV_S + 1) % MOD
            return frame
        return None

    def send_new_seq_frame(self):
        """
        Remove frame from SEQ queue
        Assign V(S) to the frame
        Insert a copy of the frame to the end of the Sent queue
        Increment V(S)
        Increment VV(S)
        Report V(S) to the I/O Sublayer
        Transfer this frame to the Frame Sublayer
        """

        """Send New SEQ frame procedure."""
        if not self.seq_queue:
            return None
        frame = self.seq_queue.popleft()
        frame.expedited = False
        frame.seq = self.V_S
        # Insert copy into Sent queue
        self.sent_queue.append(Frame(seq=frame.seq,
                                     expedited=frame.expedited,
                                     payload=frame.payload))
        self.V_S = (self.V_S + 1) % MOD
        self.VV_S = (self.VV_S + 1) % MOD
        # Report V(S) to I/O sublayer could be done via callback
        return frame

    # ----- Events on FOP‑P side (6.2.3.3) -----

    def on_SE1_need_frame(self):
        """
        SE1: Frame Sublayer needs frame to transmit.
        Returns a Frame or None if no frame available.
        Only implemented for State S1 (Active).
        """
        if self.state != FOPState.S1_ACTIVE:
            return None

        # If Expedited frame available
        if self.exp_queue:
            return self.send_exp_frame()

        # In-progress retransmission
        if less_than(self.VV_S, self.V_S):
            return self.resend_seq_frame()

        # New SEQ frame, if window allows
        if self.seq_queue and mod_diff(self.V_S, self.NN_R) < self.Transmission_Window:
            return self.send_new_seq_frame()

        # Progressive retransmission if there is any unacknowledged frame
        if less_than(self.NN_R, self.V_S):
            self.VV_S = self.NN_R
            return self.resend_seq_frame()

        # Nothing to send
        return None

    def on_SE2_valid_plcw(self, N_R, R_R):
        """
        SE2: Valid PLCW received.
        N_R and R_R are fields extracted from the received PLCW.
        """
        self.N_R = N_R
        self.R_R = R_R

        # If N(R) > NN(R), remove acknowledged frames
        if less_than(self.NN_R, self.N_R):
            self.remove_acknowledged_frames()

        # If R(R) = true or N(R) > VV(S), set VV(S) = N(R)
        if self.R_R or less_than(self.VV_S, self.N_R):
            self.VV_S = self.N_R

        self.store_plcw()
        self.clear_synch_timer()

        # If R(R) = false and N(R) = NN(R), resync complete
        if not self.R_R and equal(self.N_R, self.NN_R):
            self.RESYNC = False
            # Persistence = false; omitted here, would be managed by MAC

            # If we were in S2 and resync done, go back to S1
            if self.state == FOPState.S2_RESYNC:
                self.state = FOPState.S1_ACTIVE

    def on_SE3_invalid_plcw(self):
        """SE3: Invalid PLCW received."""
        self.start_synch_timer()
        self.VV_S = self.NN_R
        # Rest of behavior (e.g., ignore) is implicit

    def on_SE4_synch_timer_expired(self):
        """SE4: Synch‑timer expired."""
        # Notify vehicle controller would be done outside this class
        if self.Resync_Local:
            self.RR_R = False
            self.RESYNC = True
            self.state = FOPState.S2_RESYNC

    # Simple helpers to enqueue frames from upper layer

    def enqueue_expedited(self, payload: bytes):
        self.exp_queue.append(Frame(expedited=True, payload=payload))

    def enqueue_seq(self, payload: bytes):
        self.seq_queue.append(Frame(expedited=False, payload=payload))

# ---------- FARM‑P (Receiving side) ----------

@dataclass
class FARMP:
    V_R: int = 0                # Next expected SEQ (report value in PLCW)
    R_S: bool = False           # Retransmit flag for PLCW
    Expedited_Frame_Counter: int = 0  # 3‑bit counter (mod 8)

    NEED_PLCW: bool = True
    NEED_STATUS_REPORT: bool = True

    def initialize(self):
        """RE0: Initialization at session startup."""
        self.V_R = 0
        self.R_S = False
        self.Expedited_Frame_Counter = 0
        self.NEED_PLCW = True
        self.NEED_STATUS_REPORT = True

    # RE1: Invalid frame arrives
    def on_RE1_invalid_frame(self):
        # Discard frame, nothing else
        pass

    # RE2: Valid SET V(R) directive arrives
    def on_RE2_set_vr(self, seq_ctrl_fsn: int):
        self.R_S = False
        self.V_R = seq_ctrl_fsn
        self.NEED_PLCW = True

    # RE3: Valid Expedited frame arrives
    def on_RE3_valid_expedited(self, frame: Frame):
        self.Expedited_Frame_Counter = (self.Expedited_Frame_Counter + 1) % 8
        # Pass to I/O sublayer: here we just return it
        return frame

    # RE4: Valid SEQ frame, N(S) = V(R) (in sequence)
    def on_RE4_in_sequence(self, frame: Frame):
        self.R_S = False
        # Accept and pass to I/O sublayer
        out = frame
        # Increment V(R)
        self.V_R = (self.V_R + 1) % MOD
        self.NEED_PLCW = True
        return out

    # RE5: Valid SEQ frame, N(S) > V(R) (gap detected)
    def on_RE5_gap_detected(self):
        # Discard frame
        self.R_S = True
        self.NEED_PLCW = True

    # RE6: Valid SEQ frame, N(S) < V(R) (already received)
    def on_RE6_already_received(self):
        # Discard frame, nothing else
        pass

    # Classify incoming SEQ frame w.r.t. V(R)
    def classify_seq_frame(self, N_S: int) -> str:
        if equal(N_S, self.V_R):
            return "in_sequence"
        if greater_than(self.V_R, N_S):
            return "already_received"
        return "gap_detected"

    # RE7: Frame Sublayer requests PLCW contents
    def build_plcw(self):
        """
        Return PLCW contents tuple (R(S), V(R), Expedited_Frame_Counter).
        """
        self.NEED_PLCW = False
        return self.R_S, self.V_R, self.Expedited_Frame_Counter

# ---------- Simple example usage cell ----------

if __name__ == "__main__":
    # Example: basic send/receive interaction in the notebook
    fop = FOPP()
    farm = FARMP()

    fop.initialize()
    farm.initialize()

    # Enqueue two sequence‑controlled frames
    fop.enqueue_seq(b"frame0")
    fop.enqueue_seq(b"frame1")

    # Sender side: frame sublayer needs frames
    tx1 = fop.on_SE1_need_frame()
    tx2 = fop.on_SE1_need_frame()

    # Receiver side: process tx1 as in‑sequence
    cls1 = farm.classify_seq_frame(tx1.seq)
    if cls1 == "in_sequence":
        farm.on_RE4_in_sequence(tx1)

    # Build PLCW at receiver and feed back to sender
    R_S, V_R, exp_cnt = farm.build_plcw()
    fop.on_SE2_valid_plcw(N_R=V_R, R_R=R_S)

    print("Sender V(S), NN(R), Sent queue length:",
          fop.V_S, fop.NN_R, len(fop.sent_queue))
    print("Receiver V(R), R(S):", farm.V_R, farm.R_S)


Sender V(S), NN(R), Sent queue length: 2 1 1
Receiver V(R), R(S): 1 False
